In [19]:
import pandas as pd
from backtesting import Backtest, Strategy

In [20]:
Arima_df = pd.read_csv('ARIMA_csv_files/{}.csv'.format("SPY"), index_col="Date")
Arima_df = Arima_df.asfreq('D')
Arima_df

,Open,High,Low,Close,Adj Close,Volume,ARIMA_Pred,Arima_Signal
Date,,,,,,,,
2015-05-16,212.440002,212.610001,211.860001,212.440002,189.206436,76510100.0,212.335379,0.999508
2015-05-17,212.440002,212.610001,211.860001,212.440002,189.206436,76510100.0,212.621059,1.000852
2015-05-18,212.240005,213.399994,212.160004,213.100006,189.794250,74549700.0,212.770784,0.998455
2015-05-19,213.240005,213.570007,212.690002,213.029999,189.731918,72114600.0,212.739652,0.998637
2015-05-20,213.149994,213.779999,212.500000,212.880005,189.598312,76857500.0,213.014700,1.000633
...,...,...,...,...,...,...,...,...
2021-03-22,390.029999,394.070007,389.970001,392.589996,392.589996,73778600.0,392.104947,0.998764
2021-03-23,391.910004,393.459991,388.660004,389.500000,389.500000,90686600.0,394.061694,1.011712
2021-03-24,391.000000,392.750000,387.470001,387.519989,387.519989,97588600.0,390.933708,1.008809


In [21]:
period = 2017
if isinstance(period, str):
    TimeSeries_data = Arima_df.iloc[-int(float(period)*365):]
elif isinstance(period, int):
    TimeSeries_data = Arima_df.loc["{}-12-31".format(period-1):"{}-12-31".format(period),]
TimeSeries_data

,Open,High,Low,Close,Adj Close,Volume,ARIMA_Pred,Arima_Signal
Date,,,,,,,,
2016-12-31,224.729996,224.830002,222.729996,223.529999,206.632751,108998300.0,224.220751,1.003090
2017-01-01,224.729996,224.830002,222.729996,223.529999,206.632751,108998300.0,224.313732,1.003506
2017-01-02,224.729996,224.830002,222.729996,223.529999,206.632751,108998300.0,224.418880,1.003977
2017-01-03,225.039993,225.830002,223.880005,225.240005,208.213501,91366500.0,224.320925,0.995920
2017-01-04,225.619995,226.750000,225.610001,226.580002,209.452240,78744400.0,226.499405,0.999644
...,...,...,...,...,...,...,...,...
2017-12-27,267.380005,267.730011,267.010010,267.320007,251.916733,57751000.0,268.436941,1.004178
2017-12-28,267.890015,267.920013,267.450012,267.869995,252.435043,45116100.0,268.451256,1.002170
2017-12-29,268.529999,268.549988,266.640015,266.859985,251.483246,96007400.0,269.069674,1.008280


In [22]:
def get_signal(data):
    return data['Arima_Signal']

## Satoshi added new strategy on 03.28.2021
class ARIMA_Pred(Strategy):
    low = 0.98
    high = 1.02 
    long_only = 1
    
    def init(self):
        self.signal = self.I(get_signal, self.data)
            
    def next(self):
        if self.signal > self.high:  
            if self.long_only == 0:
                self.position.close()
            self.buy()

        elif self.signal < self.low:
            self.position.close()
            if self.long_only == 0:
                self.sell()

In [23]:
bt = Backtest(TimeSeries_data, ARIMA_Pred, cash=100000, commission=0)
stats = bt.run()
stats

Start                     2016-12-31 00:00:00
End                       2017-12-31 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                     62.2951
Equity Final [$]                       113907
Equity Peak [$]                        113907
Return [%]                            13.9072
Buy & Hold Return [%]                 19.3844
Return (Ann.) [%]                     13.8667
Volatility (Ann.) [%]                 5.87616
Sharpe Ratio                          2.35982
Sortino Ratio                         4.28822
Calmar Ratio                          6.66433
Max. Drawdown [%]                    -2.08073
Avg. Drawdown [%]                   -0.479943
Max. Drawdown Duration       35 days 00:00:00
Avg. Drawdown Duration        9 days 00:00:00
# Trades                                    1
Win Rate [%]                              100
Best Trade [%]                        13.9142
Worst Trade [%]                       13.9142
Avg. Trade [%]                    